In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from src.definitions import ROOT_DIR
from src.data.utils import head, line_count

from src.visualization.visualize import plot_cartesian_gridded_surface

In [ ]:
%matplotlib ipympl

In [ ]:
# Downloaded files directory
dst_dir = ROOT_DIR / "data/external/"

well_tops_path = dst_dir / "groningen/Formation_tops/Groningen__Formation_tops__EPSG_28992.csv"

In [ ]:
head(well_tops_path)

In [ ]:
line_count(well_tops_path)

In [ ]:
well_tops = pd.read_csv(well_tops_path)

In [ ]:
is_ro_top = well_tops["Surface"] == "RO____T"
ro_well_tops = well_tops.loc[is_ro_top, :]

In [ ]:
np.sum(~ro_well_tops["TWT auto"].isna())

In [ ]:
ro_well_tops = ro_well_tops.rename(columns={"X": "easting", "Y": "northing", "Z": "depth"})

In [ ]:
interp_RO_T_path = dst_dir / "groningen/Horizon_Interpretation/RO____T"
col_names = ["inline", "xline", "easting", "northing", "depth"]
ro_t = pd.read_csv(interp_RO_T_path, sep=r"\s+", header=None, names=col_names)

In [ ]:
ro_t["depth"] = -ro_t["depth"]

In [ ]:
title = "Top Rotliegend with well tops"
fig, ax = plt.subplots(figsize=(14, 14))

im = plot_cartesian_gridded_surface(
    ro_t,
    ax=ax,
    title=title,
    cmap="viridis",
    vmin=-3000,
    vmax=-2500,
    alpha=1
    )

scatter = ax.scatter(
    ro_well_tops.easting,
    ro_well_tops.northing,
    c=ro_well_tops.depth,
    cmap="viridis",
    vmin=-3000,
    vmax=-2500,
    alpha=1,
    edgecolors="red"
    )
plt.show()